# Ionization states in an interplanetary coronal mass ejection

[plasmapy.particles]: https://docs.plasmapy.org/en/stable/particles/index.html

The ionization state distribution for an element refers to the fractions of that element at each ionic level. For example, the ionization state of helium in the solar wind might be 10% He$^{0+}$, 70% He$^{1+}$, and 20% He$^{2+}$. This notebook introduces the data structures in [plasmapy.particles] for representing the ionization state of a plasma.

In [ ]:
import astropy.units as u
import matplotlib.pyplot as plt

from plasmapy.particles import IonizationState, IonizationStateCollection

## The ionization state of a single element

[IonizationState]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.ionization_state.IonizationState.html#plasmapy.particles.ionization_state.IonizationState

Let's create an [IonizationState] object for helium using the ionic fractions described above. We'll specify the total number density of helium via the `n_elem` keyword argument.

In [ ]:
He_states = IonizationState("He-4", [0.1, 0.7, 0.2], n_elem=1e13 * u.m**-3)

[ionic_fractions]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.ionization_state.IonizationState.html#plasmapy.particles.ionization_state.IonizationState.ionic_fractions
[IonizationState]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.ionization_state.IonizationState.html#plasmapy.particles.ionization_state.IonizationState

The ionization state distribution is stored in the [ionic_fractions] attribute of [IonizationState].

In [ ]:
He_states.ionic_fractions

We can get the symbols for each ionic level in `He_states` too.

In [ ]:
He_states.ionic_symbols

Because we provided the number density of the element as a whole, we can get back the number density of each ionic level.

In [ ]:
He_states.number_densities

We can also get the electron number density required to balance the positive charges for ions of this element.

In [ ]:
He_states.n_e

[IonicLevel]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.ionization_state.IonicLevel.html
[IonizationState]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.ionization_state.IonizationState.html#plasmapy.particles.ionization_state.IonizationState

We can provide an [IonizationState] with a charge number as an index to get an [IonicLevel] object that contains most of these attributes, but for a single ionic level (like He$^{1+}$).  This capability is useful if we wish to iterate over the ions of an element.

In [ ]:
for Z in range(3):
    print(He_states[Z])

We can get information about the average charge state via `Z_mean`, `Z_most_abundant`, and `Z_rms`.

In [ ]:
He_states.Z_mean

In [ ]:
He_states.Z_most_abundant

In [ ]:
He_states.Z_rms

We can calculate the properties of the average ionic level.

In [ ]:
He_states.average_ion()

We can use the `summarize()` method to get information about the ionization state.

In [ ]:
He_states.summarize()

## Ionization states of multiple elements

[*Advanced Composition Explorer*]: https://en.wikipedia.org/wiki/Advanced_Composition_Explorer
[Gilbert et al. (2012)]: https://doi.org/10.1088/0004-637X/751/1/20

Now let's look at some actual average hourly densities for ions of C, O, and Fe during an interplanetary coronal mass ejection (ICME) observed by the [*Advanced Composition Explorer*] (*ACE*) near 1 AU.  The data were estimated from Figure 4 in [Gilbert et al. (2012)].  This data set is noteworthy because there is information from very low charge states to very high charge states for several elements.

[electron ionization]: https://en.wikipedia.org/wiki/Electron_ionization
[radiative recombination]: https://en.wikipedia.org/wiki/Plasma_recombination

The [electron ionization] and [radiative recombination] rates for a given temperature are proportional to $n_i n_e$, where $n_i$ is the ion number density and $n_e$ is the electron number density.  Ionization and recombination are fast at high densities and slow at low densities. High density plasma can reach *ionization equilibrium* (when the recombination and ionization rates balance each other out) more quickly than low density plasma.  Plasma undergoing rapid heating or cooling will be in *non-equilibrium ionization* because ionization and recombination can't keep up with the temperature changes.

Quiescent plasma near the sun is typically close to ionization equilibrium because the density is high. As plasma moves away from the sun, the ionization and recombination rates drop rapidly because of the decreasing number density. The ionization states freeze out at several solar radii as the ionization and recombination time scales begin to exceed the time it takes for plasma to move from the sun to 1 AU. 

The ionization states of the solar wind are powerful diagnostics of the thermodynamic history of solar wind and ICME plasma.  The ionization states observed by *ACE* at 1 AU are essentially the same as at ∼$5R_☉$.  

In [ ]:
number_densities = {
    "C": [0, 5.7e-7, 4.3e-5, 3.6e-6, 2.35e-6, 1e-6, 1.29e-6] * u.cm**-3,
    "O": [0, 1.2e-7, 2.2e-4, 7.8e-6, 8.8e-7, 1e-6, 4e-6, 1.3e-6, 1.2e-7] * u.cm**-3,
    "Fe": [
        0,
        0,
        1.4e-8,
        1.1e-7,
        2.5e-7,
        2.2e-7,
        1.4e-7,
        1.2e-7,
        2.1e-7,
        2.1e-7,
        1.6e-7,
        8e-8,
        6.3e-8,
        4.2e-8,
        2.5e-8,
        2.3e-8,
        1.5e-8,
        3.1e-8,
        6.1e-9,
        2.3e-9,
        5.3e-10,
        2.3e-10,
        0,
        0,
        0,
        0,
        0,
    ]
    * u.cm**-3,
}

[IonizationStateCollection]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.ionization_state_collection.IonizationStateCollection.html#plasmapy.particles.ionization_state_collection.IonizationStateCollection

Let's use this information as an input for [IonizationStateCollection]: a data structure for the ionization states of multiple elements.

In [ ]:
states = IonizationStateCollection(number_densities)

We can index this to get an `IonizationState` for one of the elements.

In [ ]:
states["C"]

We can get the relative abundances of each of the elements.

In [ ]:
states.abundances

In [ ]:
states.log_abundances

We can get the number densities as a `dict` (like what we provided) and the electron number density (assuming quasineutrality, but only for the elements contained in the data structure).

In [ ]:
states.number_densities

In [ ]:
states.n_e

We can summarize this information too, but let's specify the minimum ionic fraction to print.

In [ ]:
states.summarize(minimum_ionic_fraction=0.02)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10, 3), tight_layout=True)

for state, ax in zip(states, axes):
    ax.bar(state.charge_numbers, state.ionic_fractions)
    ax.set_title(f"Ionization state for {state.base_particle}")
    ax.set_xlabel("ionic level")
    ax.set_ylabel("log$_{10}$ of ionic fraction")
    ax.set_yscale("log")

[filament]: https://en.wikipedia.org/wiki/Solar_prominence

The wide range of average ionic levels for each element is strong evidence that the plasma observed by *ACE* originated from a wide range of temperatures.  The lowest charge states are evidence of cool [filament] plasma while the high charge states are evidence of rapidly heated plasma.